#### Imports

In [3]:
import cv2
import numpy as np
import open3d as o3d
import os
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### Imports

In [4]:
def save_point_cloud(file_path, point_cloud):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    o3d.io.write_point_cloud(file_path, pcd)

def read_point_cloud(file_path):
    point_cloud = o3d.io.read_point_cloud(file_path)
    return np.asarray(point_cloud.points)

def calculate_depth_map_from_image(image):
    B = image[:, :, 0]
    G = image[:, :, 1]
    R = image[:, :, 2]
    normalized = (G + B * 256 + R * 256 * 256) / (256 * 256 - 1)
    depth_map = normalized * 1000
    return map

def mask_image(image, mask_image):
    image[mask_image == 0] = 0
    return image

def calculate_vertical_fov(image_width, image_height, fov_x):
    aspect_ratio = image_width / image_height
    fov_x_rad = np.radians(fov_x)
    fov_y_rad = 2 * np.arctan(np.tan(fov_x_rad / 2) / aspect_ratio)
    fov_y = np.degrees(fov_y_rad)
    return fov_y

def calculate_focal_length(image_width, fov_x):
    return image_width / (2 * np.tan(np.radians(fov_x) / 2))

def calculate_pixel_focal_length(image_width, image_height, fov_x, fov_y):
    fov_x_rad = np.radians(fov_x)
    fov_y_rad = np.radians(fov_y)
    fx = image_width / (2 * np.tan(fov_x_rad / 2))
    fy = image_height / (2 * np.tan(fov_y_rad / 2))
    cx = image_width / 2.0
    cy = image_height / 2.0
    return fx, fy, cx, cy

def depth_map_to_point_cloud(map, fov_x, fov_y):
    height, width = map.shape
    fx, fy, cx, cy = calculate_pixel_focal_length(width, height, fov_x, fov_y)

    point_cloud = []
    for v in range(height):
        for u in range(width):
            depth = map[v, u]
            if depth <= 0 or depth > 25:  # Skip invalid depth
                continue
            x = (u - cx) / fx
            y = (v - cy) / fy
            xy_depth = np.sqrt(np.square(x) + np.square(y))
            z = np.sqrt(np.square(depth) - np.square(xy_depth))

            point_cloud.append([x, y, z])

    # Convert to Open3D point cloud
    point_cloud_np = np.array(point_cloud)
    return point_cloud_np

#### Run post-processing

In [5]:
def clean_up_postprocessed_data(data_dir, string_to_find):
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if string_to_find in file:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Error removing {file_path}: {e}")

In [12]:
def extract_rotation_translation(T):
    # Extract rotation matrix (top-left 3x3 submatrix)
    R = T[:3, :3]
    # Extract translation vector (top-right 3x1 subvector)
    t = T[:3, 3]
    return R, t

def compute_relative_transformation(T_lidar, T_camera):
    # Compute the inverse of the lidar transformation matrix
    scaling_matrix = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, -1, 0],
        [0, 0, 0, 1]
    ])
    T_lidar = np.dot(T_lidar, scaling_matrix)
    """
    rotation_matrix = np.array([
        [np.cos(90), -np.sin(90), 0, 0],
        [np.sin(90), np.cos(90), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])
    T_lidar = np.dot(T_lidar, rotation_matrix)
    """
    T_lidar = np.linalg.inv(T_lidar)
    # Compute the relative transformation from lidar to camera
    T_lidar_to_camera = np.dot(T_lidar, T_camera)
    return T_lidar_to_camera

def transform_points(points, rotation_matrix, translation_vector):
    # Apply rotation and translation to transform points to the camera coordinate system
    transformed_points = np.dot(rotation_matrix, points.T).T + translation_vector
    return transformed_points

def filter_points_by_fov(points, transform_matrix, fov_h, fov_v):
    R, T = extract_rotation_translation(transform_matrix)
    vectors = points - T[:3]
    yaw = np.degrees(np.arctan2(transform_matrix[1, 0], transform_matrix[0, 0]))
    print(yaw)
    X = vectors[:, 0]
    Y = vectors[:, 1]
    Z = vectors[:, 2]
    alpha = np.degrees(np.radians(((np.arctan2(Y, X) * 180 / np.pi) - yaw)))
    beta = np.arctan2(Z, X) * 180 / np.pi

    mask = (np.abs(alpha) < (fov_h / 2))
    return points[mask]

def visualize_point(transform_matrix, point_cloud):
    T, R = extract_rotation_translation(transform_matrix)
    point = T[:3]
    for i in range(-10, 10, 1):
        x = point[0]
        y = point[1]
        z = point[2]
        point_cloud.append((x+i/10.0, y, z))
        point_cloud.append((x, y+i/10.0, z))
    return np.array(point_cloud)

def generate_camera_ground_truth(lidar_dir_path, camera_dir_path):
    for lidar_filename in os.listdir(lidar_dir_path):
        if lidar_filename.endswith('.ply'):
            lidar_ply_file = os.path.join(lidar_dir_path, lidar_filename)
            lidar_point_cloud = read_point_cloud(lidar_ply_file)
            lidar_npy_file = os.path.join(lidar_dir_path, lidar_filename.replace(".ply", ".npy"))
            lidar_transform_matrix = np.load(lidar_npy_file)
            camera_npy_file = os.path.join(camera_dir_path, lidar_filename.replace(".ply", ".npy"))
            camera_transform_matrix = np.load(camera_npy_file)
            
            lidar_to_camera_transform_matrix = compute_relative_transformation( 
                lidar_transform_matrix, camera_transform_matrix
            )
        
            #camera_point = lidar_transform_matrix[:, 3][:3]
            visible_points = filter_points_by_fov(lidar_point_cloud, lidar_to_camera_transform_matrix, 120.0, calculate_vertical_fov(1600, 900, 120))
            #visible_points = visualize_point(lidar_to_camera_transform_matrix, visible_points.tolist())
            #visible_points = lidar_point_cloud 
            
            visible_points_file_path = os.path.join(camera_dir_path, lidar_filename.replace(".ply", ".pointcloud.ply"))
            save_point_cloud(visible_points_file_path, visible_points)


DATA_DIR = "./generated_data"
clean_up_postprocessed_data(DATA_DIR, ".pointcloud.")

LIDAR_DIR = "LIDAR_TOP"
CAM_DIRS = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]
for CAM_DIR in CAM_DIRS:
    lidar_dir_path = os.path.join(DATA_DIR, LIDAR_DIR)
    camera_dir_path = os.path.join(DATA_DIR, CAM_DIR)
    generate_camera_ground_truth(lidar_dir_path, camera_dir_path)


Removed: ./generated_data/CAM_BACK/7484670044.pointcloud.ply
Removed: ./generated_data/CAM_BACK/9184670069.pointcloud.ply
Removed: ./generated_data/CAM_BACK/9584670075.pointcloud.ply
Removed: ./generated_data/CAM_BACK/10084670083.pointcloud.ply
Removed: ./generated_data/CAM_BACK/7184670039.pointcloud.ply
Removed: ./generated_data/CAM_BACK/8884670065.pointcloud.ply
Removed: ./generated_data/CAM_BACK/10884670095.pointcloud.ply
Removed: ./generated_data/CAM_BACK/8784670063.pointcloud.ply
Removed: ./generated_data/CAM_BACK/10684670092.pointcloud.ply
Removed: ./generated_data/CAM_BACK/7784670048.pointcloud.ply
Removed: ./generated_data/CAM_BACK/9984670081.pointcloud.ply
Removed: ./generated_data/CAM_BACK/10584670090.pointcloud.ply
Removed: ./generated_data/CAM_BACK/6784670033.pointcloud.ply
Removed: ./generated_data/CAM_BACK/8684670062.pointcloud.ply
Removed: ./generated_data/CAM_BACK/10384670087.pointcloud.ply
Removed: ./generated_data/CAM_BACK/7684670047.pointcloud.ply
Removed: ./generate

In [14]:
def combine_ply_files_with_transformations(directory, output_file):
    combined_points = []
    combined_colors = []

    # Iterate over all files in the directory
    for i, filename in enumerate(os.listdir(directory)):
        if filename.endswith('.ply'):
            ply_path = os.path.join(directory, filename)
            npy_path = os.path.join(directory, filename.replace('.ply', '.npy').replace('.pointcloud', ''))

            if not os.path.exists(npy_path):
                print(f"Warning: No matching .npy file for {filename}")
                continue

            # Load the PLY file
            ply = o3d.io.read_point_cloud(ply_path)
            points = np.asarray(ply.points)
            colors = np.asarray(ply.colors) if ply.has_colors() else None

            # Load the transformation matrix
            transformation_matrix = np.load(f"{DATA_DIR}/{LIDAR_DIR}/{filename.replace('.ply', '.npy').replace('.pointcloud', '')}") #np.load(npy_path)
            

            # Apply the transformation
            points_homogeneous = np.hstack((points, np.ones((points.shape[0], 1))))
            transformed_points = (transformation_matrix @ points_homogeneous.T).T[:, :3]

            combined_points.append(transformed_points)
            if colors is not None:
                combined_colors.append(colors)
            if i > 20:
                break
            

    # Combine all points and colors
    all_points = np.vstack(combined_points)
    if combined_colors:
        all_colors = np.vstack(combined_colors)
    else:
        all_colors = None

    # Create a new PointCloud object for the combined points
    combined_ply = o3d.geometry.PointCloud()
    combined_ply.points = o3d.utility.Vector3dVector(all_points)
    if all_colors is not None:
        combined_ply.colors = o3d.utility.Vector3dVector(all_colors)

    # Write the combined PLY to a file
    o3d.io.write_point_cloud(output_file, combined_ply)
    print(f"Combined PLY file written to {output_file}")

# Example usage
for cam_dir in ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]:
    combine_ply_files_with_transformations(f'generated_data/{cam_dir}', f'generated_data/BEV/{cam_dir.lower()}_combined_point_cloud.ply')


Combined PLY file written to generated_data/BEV/cam_front_combined_point_cloud.ply
Combined PLY file written to generated_data/BEV/cam_front_left_combined_point_cloud.ply
Combined PLY file written to generated_data/BEV/cam_front_right_combined_point_cloud.ply
Combined PLY file written to generated_data/BEV/cam_back_combined_point_cloud.ply
Combined PLY file written to generated_data/BEV/cam_back_left_combined_point_cloud.ply
Combined PLY file written to generated_data/BEV/cam_back_right_combined_point_cloud.ply


In [70]:
import open3d as o3d
import numpy as np

# Create a cylinder
mesh_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=1.0, height=2.0)
mesh_cylinder.compute_vertex_normals()

# Convert mesh to point cloud
point_cloud = mesh_cylinder.sample_points_poisson_disk(number_of_points=2500)

o3d.io.write_point_cloud(f"{DATA_DIR}/DEMO_DATA/demo.ply", point_cloud)


True

In [14]:
points = read_point_cloud(f"{DATA_DIR}/DEMO_DATA/demo.ply")
for i in range(100):
    filtered = filter_points_by_fov(points, [i/10.0,0,0], 120.0, calculate_vertical_fov(1600, 900, 120))
    save_point_cloud(f"{DATA_DIR}/DEMO_DATA/{i}.ply", filtered)

[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
[Open3D WARNING] Write P

In [6]:
DATA_DIR = "./generated_data"
SEMANTIC_CAM_DIRS = ["SEMANTIC_CAM_FRONT", "SEMANTIC_CAM_FRONT_LEFT", "SEMANTIC_CAM_FRONT_RIGHT", "SEMANTIC_CAM_BACK", "SEMANTIC_CAM_BACK_LEFT", "SEMANTIC_CAM_BACK_RIGHT"]
DEPTH_CAM_DIRS = ["DEPTH_CAM_FRONT", "DEPTH_CAM_FRONT_LEFT", "DEPTH_CAM_FRONT_RIGHT", "DEPTH_CAM_BACK", "DEPTH_CAM_BACK_LEFT", "DEPTH_CAM_BACK_RIGHT"]
fov_w_degrees, fov_h_degrees = 120.0, calculate_vertical_fov(1600, 900, 120)

for (SEM_SUBDIR, DEPTH_SUBDIR) in zip(SEMANTIC_CAM_DIRS, DEPTH_CAM_DIRS):
    directory = f"{DATA_DIR}/{DEPTH_SUBDIR}"
    for lidar_filename in os.listdir(directory):
        if lidar_filename.endswith('.png'):
            timestamp = lidar_filename.replace(".png", "")
            depth_image_path = os.path.join(directory, lidar_filename)
            image = cv2.imread(depth_image_path)
            depth_map = calculate_depth_map_from_image(image)
            point_cloud = depth_map_to_point_cloud(depth_map, fov_w_degrees, fov_h_degrees)
            save_point_cloud(f"{DATA_DIR}/{DEPTH_SUBDIR}/{timestamp}.pointcloud.ply", point_cloud)
            depth_image_mask = cv2.imread(f"{DATA_DIR}/{SEM_SUBDIR}/{timestamp}.mask.png", cv2.IMREAD_GRAYSCALE)
            masked_depth_image = mask_image(image, depth_image_mask)
            depth_map = calculate_depth_map_from_image(masked_depth_image)
            point_cloud = depth_map_to_point_cloud(depth_map, fov_w_degrees, fov_h_degrees)
            save_point_cloud(f"{DATA_DIR}/{DEPTH_SUBDIR}/{timestamp}.pointcloud.masked.ply", point_cloud)

NameError: name 'depth_map_to_point_cloud' is not defined